In [ ]:
import cv2
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import joblib


data = []
labels = []

path = "Dataset"   

classes = ["With-Mask", "Without-Mask"]

for label, i in enumerate(classes):
    folder = os.path.join(path, i)
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path)

        if img is None:
            continue

        img = cv2.resize(img, (50, 50))        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = img.flatten()                    

        data.append(img)
        labels.append(label)

data = np.array(data)
labels = np.array(labels)


X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=42
)

model = SVC(kernel='linear', probability=True)
model.fit(X_train, y_train)


pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, pred))

joblib.dump(model, "mask_model.pkl")


Accuracy: 0.8222222222222222
Model saved as mask_model.pkl


In [ ]:
import cv2
import numpy as np
import joblib

model = joblib.load("mask_model.pkl")

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (50, 50))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        face = face.flatten().reshape(1, -1)



        pred = model.predict(face)[0]
        label = "With Mask" if pred == 0 else "No Mask"
        color = (0, 255, 0) if pred == 0 else (0, 0, 255)

        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)


    cv2.imshow("Mask Detector" , frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()
